This is an example of crossmatching Gaia DR2 objects with Pan-STARRS 1 photometry

In [1]:
import PSMatch as PSMatch

import photConv as photConv
from astroquery.vizier import Vizier #bugged in astropy 4.0+
import astropy.units as unitz
from astropy.coordinates import Angle,SkyCoord,Distance
from astropy.time import Time
import numpy as np

gzip was not found on your system! You should solve this issue for astroquery.eso to be at its best!
On POSIX system: make sure gzip is installed and in your path!On Windows: same for 7-zip (http://www.7-zip.org)!


In [27]:
#fonction de recherche dans ps1
def gaia2rSDSS(G,BPRP):
    #https://gea.esac.esa.int/archive/documentation/GDR2/Data_processing/chap_cu5pho/sec_cu5pho_calibr/ssec_cu5pho_PhotTransf.html
    #retourne r SDSS
    return G -0.12879 +0.24662*BPRP -0.027464*BPRP**2 -0.049465*BPRP**3

def convert_epoch(RA,DEC,pmra,pmdec,parallax):
    #converts J2015.5 to J2000
    RA = Angle(RA+' deg') #change this to deg later
    DEC = Angle(DEC+' deg')
    dist = Distance(1000/float(parallax),unitz.pc)
    pmra = float(pmra)* unitz.mas/unitz.yr
    pmdec = float(pmdec)* unitz.mas/unitz.yr
    print(pmra,pmdec)
    
    c = SkyCoord(ra=RA,
                 dec=DEC,
                 distance=dist,
                 pm_ra_cosdec=pmra,
                 pm_dec=pmdec,
                 obstime=Time('J2015.5', format='jyear_str'))
    
    j2000 = c.apply_space_motion(new_obstime=Time('J2000.0', format='jyear_str'))
    
    return j2000.ra.degree,j2000.dec.degree

""" ------------------------------------------------------------------------------------------- """
def find_ps1(gaiainfo=[],SDSSPhot=[],searchR=0.00139):
    gaianame,gaiaRA,gaiaDEC,p,pe,gaiapmRA,gaiapmDEC,G,BP,RP,BR=gaiainfo
    uSDSS,gSDSS,rSDSS,iSDSS,zSDSS=[float(i) for i in SDSSPhot]
    G = float(G)
    BR = float(BR)
    
    flags,info=[False,'-'],['-']*13
    
    #convert epoch here
    # PS1 DR1 is in J2000 in Vizier, whereas Gaia DR2 is in J2015.5
    RA,DEC = convert_epoch(gaiaRA,gaiaDEC,gaiapmRA,gaiapmDEC,p)
    #print(RA,DEC)
    
    #cherche par coord avec searchR sec d'arc
    results=Vizier.query_region(SkyCoord(ra=RA, dec=DEC,unit=(unitz.deg, unitz.deg),frame='icrs'),
                         radius=Angle(searchR,'deg'),
                         catalog='II/349/ps1')
    if len(results)>0: flags,info=PS1search_v2(results,RA,DEC,G,BR,SDSSPhot)
    
    #plus grand rayon
    else:
        results=Vizier.query_region(SkyCoord(ra=RA, dec=DEC,unit=(unitz.deg, unitz.deg),frame='icrs'),
                         radius=Angle(searchR*4,'deg'),
                         catalog='II/349/ps1')
        if len(results)>0: flags,info=PS1search_v2(results,RA,DEC,G,BR,SDSSPhot)
    
    return flags,info

""" ------------------------------------------------------------------------------------------- """
def PS1search(table,RA,DEC,G,BR,gSDSS,maxMagDiff=0.3):
    table=table[0]
    
    #default values
    ps1matched=False
    ps1meth='-'
    name,ra,dec,g,ge,r,re,i,ie,z,ze,y,ye=['-']*13
    
    RA_all=table['RAJ2000']
    DEC_all=table['DEJ2000']
    g_all=table['gmag']
    
    #si on a un g
    if gSDSS!='00.00':
        #cherche l'objet avec le g le plus proche
        idx=(np.abs(g_all - gSDSS)).argmin()
        if np.abs(gSDSS-g_all[idx])<maxMagDiff:
            ps1matched=True
            ps1meth='gSDSS'
            name,ra,dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
    
    #sinon on convertit G gaia en g SDSS
    if not ps1matched:
        g_approx=photConv.gaia2SDSS(G,BR)
        idx=(np.abs(g_all - g_approx)).argmin()
        if np.abs(g_approx-g_all[idx])<maxMagDiff:
            ps1matched=True
            ps1meth='GaiaG'
            name,ra,dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
    
    return [ps1matched,ps1meth],[name,ra,dec,g,ge,r,re,i,ie,z,ze,y,ye]

""" ------------------------------------------------------------------------------------------- """
def PS1search_v2(table,RA,DEC,G,BPRP,tabSDSS):
    # fouille une table vizier pour le meilleur ps1 xmatch
    table=table[0]
    
    #default values
    ps1matched=False
    ps1meth='-'
    ps1name,ps1ra,ps1dec=['-']*3
    g,ge,r,re,i,ie,z,ze,y,ye=['-']*10
    
    #export SDSS grz
    gSDSS,rSDSS,zSDSS=float(tabSDSS[1]),float(tabSDSS[2]),float(tabSDSS[4])
    
    RA_all=table['RAJ2000']
    DEC_all=table['DEJ2000']
    g_all=table['gmag']
    r_all=table['rmag']
    z_all=table['zmag']
    
    #si on a ugriz
    if gSDSS>0.:
        #cherche l'objet avec grz le plus proche
        #on fait une moyenne des diff entre les filtres
        g_diff=np.abs(g_all - gSDSS)
        r_diff=np.abs(r_all - rSDSS)
        z_diff=np.abs(z_all - zSDSS)
        avg_diff = (g_diff+r_diff+z_diff)/3
        
        #prend la difference moyenne la plus petite
        idx=avg_diff.argmin()
        if (np.abs(gSDSS-g_all[idx])<0.3)and(np.abs(rSDSS-r_all[idx])<0.3)and(np.abs(zSDSS-z_all[idx])<0.3):
            ps1matched=True
            ps1meth='gSDSS'
            ps1name,ps1ra,ps1dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
        else:
            print('PS1-SDSS xmatch failed:',gSDSS,g_all[idx],zSDSS,z_all[idx])
    
    #sinon on approxime un rSDSS avec valeurs Gaia
    if not ps1matched:
        r_approx=gaia2rSDSS(G,BPRP)
        idx=(np.abs(r_all - r_approx)).argmin()
        if np.abs(r_approx-r_all[idx])<0.7:
            ps1matched=True
            ps1meth='approx_rSDSS'
            ps1name,ps1ra,ps1dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
            
    #the gaia->sdss convertion fails for some bright stars
    #i use an arbitrary 14 mag limit
    #for this case, use the Gaia mag as is
    if (G<14.)and(not ps1matched):
        idx=(np.abs(g_all - G)).argmin()
        if np.abs(G-g_all[idx])<1.:
            ps1matched=True
            ps1meth='bright_Gmag'
            ps1name,ps1ra,ps1dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
            
    #as last desperate check for bright stars
    #if it is VERY close and alone in the search radius, take it  
    if (G<14.)and(not ps1matched)and(len(table)==1):
        limDeg = 0.00138889 # 5deg lim
        idx=0    
        if (np.abs(RA-RA_all[idx])<limDeg)and(np.abs(DEC-DEC_all[idx])<limDeg):
            ps1matched=True
            ps1meth='bright_dist'
            ps1name,ps1ra,ps1dec=table['objID'][idx],table['RAJ2000'][idx],table['DEJ2000'][idx]
            g,ge=table['gmag'][idx],table['e_gmag'][idx]
            r,re=table['rmag'][idx],table['e_rmag'][idx]
            i,ie=table['imag'][idx],table['e_imag'][idx]
            z,ze=table['zmag'][idx],table['e_zmag'][idx]
            y,ye=table['ymag'][idx],table['e_ymag'][idx]
    
    return [ps1matched,ps1meth],[ps1name,ps1ra,ps1dec,g,ge,r,re,i,ie,z,ze,y,ye]

In [57]:
gaia_sample = np.loadtxt('gaia.csv',delimiter=',',dtype=str)

In [58]:
gaia_sample[0]

array(['designation', 'source_id', 'ra', 'dec', 'parallax',
       'parallax_error', 'pmra', 'pmdec', 'phot_g_mean_mag',
       'phot_bp_mean_mag', 'phot_rp_mean_mag', 'bp_rp', 'su', 'sg', 'sr',
       'si', 'sz'], dtype='<U28')

In [59]:
newfile = open('ps1matches.txt','w')

for i in range(1,len(gaia_sample)):
    obj = gaia_sample[i]
    if i%50==0: print('{}/{}...'.format(i,len(gaia_sample)),end='')
    
    test1,test2=find_ps1(obj[1:12],obj[12:])
    #print(obj[0],test2)
    #print(test1)
    
    newfile.write(obj[0]+',')
    for item in test2[3:]+test1:
        newfile.write(str(item)+',')
    newfile.write('\n')
newfile.close()

50/9369...PS1-SDSS xmatch failed: 18.652 15.3759 15.85771 15.6591
PS1-SDSS xmatch failed: 18.283 18.1647 18.697 17.2487
100/9369...150/9369...200/9369...PS1-SDSS xmatch failed: 20.393 20.3399 21.177 21.619
250/9369...300/9369...PS1-SDSS xmatch failed: 12.856 21.7346 13.581 20.5875
PS1-SDSS xmatch failed: 14.1 14.2131 0.0 15.2476
350/9369...400/9369...PS1-SDSS xmatch failed: 18.594 18.152 17.317 17.527
450/9369...500/9369...PS1-SDSS xmatch failed: 15.65 15.6491 0.0 16.1057
550/9369...PS1-SDSS xmatch failed: 14.908 13.6638 14.336 14.3211
600/9369...PS1-SDSS xmatch failed: 20.263 -- 19.018 21.1536
PS1-SDSS xmatch failed: 19.641 19.4597 19.463 18.7609
PS1-SDSS xmatch failed: 19.29 19.2148 19.603 18.879
PS1-SDSS xmatch failed: 13.098 12.9451 13.886 12.8283
650/9369...PS1-SDSS xmatch failed: 18.933 19.1692 18.384 18.6353
PS1-SDSS xmatch failed: 20.01783 19.9592 19.81201 19.4519
700/9369...PS1-SDSS xmatch failed: 18.648 18.1157 22.827 18.1802
PS1-SDSS xmatch failed: 18.085 17.874 18.405 17.79

PS1-SDSS xmatch failed: 20.836 18.1392 19.415 17.3715
7750/9369...7800/9369...PS1-SDSS xmatch failed: 17.513 17.5589 18.382 18.0366
7850/9369...7900/9369...PS1-SDSS xmatch failed: 13.715 18.4997 13.773 17.5817
7950/9369...PS1-SDSS xmatch failed: 16.847 18.6845 22.827 20.1109
PS1-SDSS xmatch failed: 19.454 19.3194 22.827 18.9169
8000/9369...PS1-SDSS xmatch failed: 15.872 18.7714 15.649 17.88
8050/9369...8100/9369...8150/9369...PS1-SDSS xmatch failed: 13.15 13.2267 0.0 14.2144
8200/9369...8250/9369...8300/9369...PS1-SDSS xmatch failed: 16.042 17.379 14.954 16.5811
8350/9369...PS1-SDSS xmatch failed: 21.577 21.2172 19.233 19.2137
PS1-SDSS xmatch failed: 17.97 -- 16.49 20.3152
8400/9369...8450/9369...8500/9369...PS1-SDSS xmatch failed: 15.711 13.0597 13.326 13.1034
8550/9369...PS1-SDSS xmatch failed: 19.67399979 19.1771 18.72299957 18.7702
8600/9369...PS1-SDSS xmatch failed: 15.682 14.4383 13.663 13.6957
8650/9369...8700/9369...PS1-SDSS xmatch failed: 15.83 15.8362 16.824 16.4442
8750/9369

227.4389595919335 45.77344932866076


In [30]:
test1,test2 = convert_epoch('12.29656845','5.376933285',1231.325249,-2711.830089,231.7374995)

1231.325249 mas / yr -2711.830089 mas / yr


In [31]:
test1,test2

(12.291243377511439, 5.3886091968322045)